In [1]:
import numpy as np 
import pandas as pd
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from keras.wrappers.scikit_learn import KerasClassifier

#Library to use spotify for getting song features
import spotipy
import time
from IPython.core.display import clear_output
from spotipy import SpotifyClientCredentials, util

In [2]:
def build_model():
    model = load_model('../Models/MER.h5')
    return model

classifier = KerasClassifier(build_fn=build_model,epochs=300,batch_size=200,verbose=0)
classifier.model = build_model()
classifier.classes_ = np.array([0, 1, 2, 3])

In [3]:
df = pd.read_csv('../Datasets/Spotify Data.csv')
Y = df['mood']

encoder = LabelEncoder()
encoder.fit(Y)
encoded_y = encoder.transform(Y)

target = pd.DataFrame({'mood':df['mood'].tolist(),'encode':encoded_y}).drop_duplicates().sort_values(['encode'],ascending=True)
target

,mood,encode
2,Calm,0
0,Energetic,1
1,Happy,2
5,Sad,3


In [4]:
client_id='5bb8ade946c2454f8262474d0040f9c2'
client_secret='1cb00aa57c334fa2aca7a54abf987d6a'

#Credentials to access the Spotify Music Data
manager = SpotifyClientCredentials(client_id,client_secret)
sp = spotipy.Spotify(client_credentials_manager=manager)

def get_songs_features(ids):

    meta = sp.track(ids)
    features = sp.audio_features(ids)

    # meta
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']
    ids =  meta['id']

    # features
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    valence = features[0]['valence']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    key = features[0]['key']
    time_signature = features[0]['time_signature']

    track = [name, album, artist, ids, release_date, popularity, length, danceability, acousticness,
            energy, instrumentalness, liveness, valence, loudness, speechiness, tempo, key, time_signature]
    columns = ['name','album','artist','id','release_date','popularity','length','danceability','acousticness','energy','instrumentalness',
                'liveness','valence','loudness','speechiness','tempo','key','time_signature']
    return track,columns

def get_track_id(track, artist, album):
    
    search_result = sp.search(q='artist:' + artist + ' track:' + track + ' album:' + album, type='track')
    track_id = search_result['tracks']['items'][0]['id']
    return track_id

In [5]:
def predict_mood(id_song):

    #Obtain the features of the song
    preds = get_songs_features(id_song)

    #Pre-process the features to input the Model
    preds_features = np.array(preds[0][6:-2]).reshape(-1,1).T

    #Predict the features of the song
    results = classifier.predict(preds_features)

    mood = np.array(target['mood'][target['encode']==int(results)])
    name_song = preds[0][0]
    artist = preds[0][2]

    return print("{0} by {1} is a {2} song".format(name_song,artist,mood[0].upper()))
    #print(f"{name_song} by {artist} is a {mood[0].upper()} song")

In [7]:
track = 'Shape Of You'
artist = 'Ed Sheeran'
album = '÷'

track_id = get_track_id(track, artist, album)
predict_mood(track_id)

Shape of You by Ed Sheeran is a ENERGETIC song
